Task1_0829. 가상 데이터 생성 (generate_data 함수) 후 모델링 및 평가하세요
- generate_data 함수는 1000개의 랜덤 시퀀스 데이터를 생성합니다.
  - vocab_size: 시퀀스에 사용할 어휘의 크기를 설정합니다. 여기서는 100개의 단어를 사용합니다.
  - data: 각 시퀀스는 seq_length=10으로 설정된 10개의 정수(단어 인덱스)로 구성됩니다.
  - labels: 각 시퀀스에 대해 0 또는 1의 이진 레이블을 무작위로 할당합니다.

- LSTM 기반 분류 모델을 정의하고, 가상 데이터로 학습 및 검증을 수행합니다.
- 조기 종료를 통해 학습 중 성능이 더 이상 개선되지 않을 때 학습을 중단합니다.
최종적으로 테스트 데이터로 최고 성능의 모델을 평가합니다.

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split


# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels


data, labels = generate_data()
print(data.shape, labels.shape)
for i in range(10):
    print(data[i], labels[i])

torch.Size([1000, 10]) torch.Size([1000])
tensor([55, 56, 44, 47, 94, 52, 55, 99, 18,  4]) tensor(0)
tensor([20, 85, 40, 98, 29, 54, 70, 26, 17, 32]) tensor(1)
tensor([57, 22, 91, 64, 92, 14, 77, 78, 23, 59]) tensor(1)
tensor([43, 11, 93, 43, 71,  2, 87, 45, 63, 73]) tensor(0)
tensor([59, 68, 69, 28, 74, 43, 62, 57, 31, 27]) tensor(1)
tensor([68, 84, 41, 93, 31, 28, 11, 35, 44, 25]) tensor(1)
tensor([94, 94, 91, 31, 81, 70, 63, 81, 83, 98]) tensor(1)
tensor([27, 98, 53, 26, 38, 76, 74, 92, 31, 93]) tensor(0)
tensor([90, 13, 12, 16, 28, 32, 83, 80, 61, 11]) tensor(0)
tensor([ 2, 53,  0, 24, 77, 83, 69, 40, 83, 87]) tensor(1)


In [7]:
# concatenate data and labels
dataset = TensorDataset(data, labels)
dataset_size = len(dataset)
dataset_size

1000

In [15]:
type(dataset)

torch.utils.data.dataset.TensorDataset

In [23]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [700, 200, 100])

trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [24]:
# lstm model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)

        out = self.embedding(x)
        out, _ = self.lstm(out, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [25]:
model = LSTM(vocab_size=100, embedding_dim=32, hidden_dim=100, num_layers=2, num_classes=2).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# train
best_val_loss = float("inf")  # 줄어드는지 확인하기 위해 가장 큰값으로 셋팅
patience, trials = 5, 0
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for input, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(input)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # validation
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for input, labels in valloader:
            outputs = model(input)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(valloader)
    print(
        f"Epoch:{epoch+1}, Training Loss: {running_loss/len(trainloader)}, Validation Loss: {val_loss}"
    )

    # 조기 종료를 포함한 모델 학습 및 best model 저장
    if val_loss < best_val_loss:
        print(
            f"validation loss decreased from {best_val_loss:.6f} to {val_loss:.6f}. Saving the model..."
        )
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), "best_model_fashion.pth")
    else:
        trials += 1
        if trials >= patience:
            print(f"early stop on epoch {epoch+1}")
            break

# load weight
model.load_state_dict(torch.load("best_model_fashion.pth", weights_only=True))

# evaluation on testloader
correct, total = 0, 0
model.eval()
with torch.no_grad():
    for input, labels in testloader:
        outputs = model(input)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

Epoch:1, Training Loss: 0.6948290196332064, Validation Loss: 0.6947515521730695
validation loss decreased from inf to 0.694752. Saving the model...
Epoch:2, Training Loss: 0.6939650638536974, Validation Loss: 0.6948898094041007
Epoch:3, Training Loss: 0.6935347589579496, Validation Loss: 0.6947405338287354
validation loss decreased from 0.694752 to 0.694741. Saving the model...
Epoch:4, Training Loss: 0.6931178542700681, Validation Loss: 0.694702787058694
validation loss decreased from 0.694741 to 0.694703. Saving the model...
Epoch:5, Training Loss: 0.6932640427892859, Validation Loss: 0.6948157548904419
Epoch:6, Training Loss: 0.6933223496783864, Validation Loss: 0.6946726271084377
validation loss decreased from 0.694703 to 0.694673. Saving the model...
Epoch:7, Training Loss: 0.6929137815128673, Validation Loss: 0.6946959836142403
Epoch:8, Training Loss: 0.6935121010650288, Validation Loss: 0.6946798392704555
Epoch:9, Training Loss: 0.6931159008633007, Validation Loss: 0.69531724282